## Data Importation/Basinc Exploration

In [1]:
# Import panads 
import pandas as pd

In [ ]:
# Downlaod data

#!wget -N https://github.com/fiddler-labs/p2p-lending-data/raw/refs/heads/master/raw_data/accepted_2007_to_2018Q3.csv.gz

# If wget doesn't work use the code below

import requests

# URL of the dataset
url = "https://github.com/fiddler-labs/p2p-lending-data/raw/refs/heads/master/raw_data/accepted_2007_to_2018Q3.csv.gz"

# Local file path to save the downloaded file
file_path = "accepted_2007_to_2018Q3.csv.gz"

# Download the file
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"File downloaded successfully and saved as {file_path}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


In [ ]:
# Set DF
df = pd.read_csv('accepted_2007_to_2018Q3.csv.gz')

In [ ]:
# Display Columns 
pd.set_option('display.max_columns', None)
print(df.columns.tolist())

In [ ]:
# Group by Loan Status
df.groupby(['loan_status']).size()


In [ ]:
# Loan Status to Charged Off/Fully Paid then Group by Size
df = df[df['loan_status'].isin(['Charged Off', 'Fully Paid'])]
df.groupby(['loan_status']).size()


## Data Cleaning on Subset of DF
- Reduce dataset to 200k rows or less
- Split your data for train/test
- Build at least two models with loan_status as the target
- Report on the models performance
- Report on the models drivers (feature importance / effect)

Plant 2 flaws in this notebook.  Do not "give away" the answers here.  This notebook should appear, at first glance, to create a viable model.

Make a note of your flaws and be prepared to talk about how the code / results look when they are "fixed".  You will need to talk about those in the final presentation.



In [ ]:
# Import everything else 

import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

from IPython.display import clear_output
from datetime import datetime

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

%matplotlib inline
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style('ticks')

from os import listdir
from os.path import isfile, join
import warnings
warnings.filterwarnings('ignore')

There are over a million rows in the dataset. For processing purposes, we are going to work on a random sample of this large dataset. 

In [ ]:
loan_df = df.sample(frac=.05, random_state=0)

print('Total Number of Rows:', '{:,}'.format(loan_df.shape[0]))
print('Total Number of Columns:', '{:,}'.format(loan_df.shape[1]))

Missing Values: Visualization and Feature Selection checking for and cleaning missing data. A visual check at the data set shows there are several columns with a large number of null values. 

In [ ]:
def get_missing_values_table(df):
    #Number of null values by column
    missing_values_df = pd.DataFrame(df.isnull().sum(),columns=['Missing_Count'])
    
    #Portion of null values by column
    missing_values_df['Portion_Missing'] = missing_values_df['Missing_Count']/df.shape[0]

    #Sort by Missing_Count
    missing_values_df = missing_values_df.sort_values(by='Missing_Count',ascending=False)  
    
    return missing_values_df

missing_values_df = get_missing_values_table(loan_df)
missing_values_df.head()

In [ ]:
print('Number of Columns with under 15% missing values:', 
      missing_values_df[missing_values_df['Portion_Missing']<=0.15].shape[0])

#Plot the distribution of Portions of missing values for the columns
fig, ax = plt.subplots(figsize=(5,5))
sns.distplot(missing_values_df['Portion_Missing'], bins=10, kde=False, rug=True)
ax.set_title('Distribution of Portion of Missing Values')
ax.set_xlabel('Portion of Missing Values for Columns in Dataset')
ax.set_ylabel('Count')
sns.despine()

Upon first glance, we see that there are many columns that do have a reasonably low level of missingess: 93 columns that have less than 15% missing values.

There seems to be a small concentration of columns between 60-70% missing values. Below, we explore this and see that there are 11 columns with almost exactly the same number of missing values. Furthermore, using the issue_d column (which indicates the date the date in which the loan was issued), we found that these variables had missing values for all loans issued prior to 2015-12-01. It's possible that the Lending Club simply did not record or use this particular information from potential borrowers until a later date. We have chosen not to include these columns because:

there are many other columns that capture similar credit-related information; and

we want to keep the older loan information in the dataset, especially since we are looking at only completed loans. Removing the older samples would greatly reduce our sample size.

In [ ]:
missing_grp = missing_values_df[(missing_values_df['Portion_Missing']>=0.6)&(missing_values_df['Portion_Missing']<=0.7)]
display(missing_grp)

missing_columns = missing_grp.index

earliest_date = []
for column in missing_columns:
    earliest_date.append(min(loan_df[~loan_df[column].isnull()]['issue_d'])) 

display(pd.DataFrame({'Column':missing_columns,
                      'Earliest issue_d for which column value is not null':earliest_date}))

# Data Cleaning: Column Removal and Selection

## 1. Identification Columns
- **Dropped Columns**: 
  - `id`, `member_id`, `url`
  - **Reason**: These columns contain 100% missing values and are identifiers, which are not relevant to the model.
  - `next_pymnt_d`: Also has 100% missing values because these loans are completed and no next payment date exists.



## 2. Single-Value Columns
- **Dropped Columns**:
  - `pymnt_plan`, `out_prncp`, `out_prncp_inv`, `policy_code`
  - **Reason**: These columns contain only one unique value, offering no variance or predictive power.



## 3. Columns with Large Missingness
- **Dropped Columns**:
  - `open_acc_6m`, `open_act_il`, `open_il_12m`, `open_il_24m`, `total_bal_il`, `open_rv_12m`, `open_rv_24m`, `max_bal_bc`, `inq_fi`, `total_cu_tl`, `inq_last_12m`
  - **Reason**: These columns have a large number of missing values, primarily for loans issued prior to April 2016. Retaining these would greatly reduce the dataset size, so they were excluded. Similar information is captured by other columns.
  - `il_util`, `all_util`, `mths_since_rcnt_il`: Slightly more missing data but followed the same trend and were removed for the same reasons.



## 4. Hardship-Related Variables
- **Dropped Columns**:
  - `deferral_term`, `hardship_amount`, `hardship_dpd`, `hardship_end_date`, `hardship_flag`, `hardship_last_payment_amount`, `hardship_length`, `hardship_loan_status`, `hardship_payoff_balance_amount`, `hardship_reason`, `hardship_start_date`, `hardship_status`, `hardship_type`, `orig_projected_additional_accrued_interest`, `payment_plan_start_date`
  - **Reason**: These columns pertain to borrowers on hardship plans. Since the information is coded as missing for borrowers not on hardship plans and would not be known at loan origination, they were excluded.



## 5. Borrower-Provided Text Fields
- **Dropped Columns**:
  - `desc`, `title`, `emp_title`
  - **Reason**: These are string variables with borrower-provided text. Equivalent categorical variables exist that capture similar information.



## 6. Settlement-Related Variables
- **Dropped Columns**:
  - `debt_settlement_flag`, `debt_settlement_flag_date`, `settlement_amount`, `settlement_date`, `settlement_percentage`, `settlement_status`, `settlement_term`, `recoveries`, `collection_recovery_fee`
  - **Reason**: These variables relate to specifics of loan settlement, which would not be known at loan origination and are not helpful for predictive modeling.



## 7. Lending Club Loan-Specific Information
- **Dropped Columns**:
  - `funded_amnt`, `funded_amnt_inv`, `last_pymnt_amnt`, `out_prncp`, `out_prncp_inv`, `total_pymnt`, `total_pymnt_inv`, `total_rec_int`, `total_rec_late_fee`, `total_rec_prncp`, `last_pymnt_d`, `last_credit_pull_d`, `disbursement_method`
  - **Reason**: These variables provide information about loan specifics (e.g., late fees, payments made) that would not be known at loan origination.



## 8. FICO-Related Columns
### Dropped SEC FICO Columns:
- **Dropped Columns**:
  - `sec_app_fico_range_high`, `sec_app_fico_range_low`
  - **Reason**: These columns contain a significant number of missing values.

### Dropped Last FICO Columns:
- **Dropped Columns**:
  - `last_fico_range_high`, `last_fico_range_low`
  - **Reason**: Represent the borrower’s most recent FICO score, which might have been updated after the loan was issued. Including these could lead to **data leakage**.

### Included High/Low FICO Columns:
- **Kept Columns**:
  - `fico_range_high`, `fico_range_low`
  - **Reason**: Represent the borrower’s FICO score **at loan origination**, which is critical for predicting loan performance. 
  - **Note**: These columns are perfectly correlated. To avoid redundancy, only `fico_range_high` was retained.


In [ ]:
exclude = ['id', 'member_id', 'url', 'next_pymnt_d', 'pymnt_plan', 'out_prncp',
           'out_prncp_inv', 'policy_code', 'open_acc_6m', 'open_act_il', 'open_il_12m',
           'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
           'inq_fi', 'total_cu_tl', 'inq_last_12m', 'il_util', 'all_util','mths_since_rcnt_il',
           'num_tl_120dpd_2m', 'num_tl_30dpd', 'deferral_term', 'hardship_amount',
           'hardship_dpd', 'hardship_end_date', 'hardship_flag', 'hardship_last_payment_amount',
           'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount',
           'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type',
           'orig_projected_additional_accrued_interest', 'payment_plan_start_date',
           'desc', 'title', 'emp_title', 'debt_settlement_flag', 'debt_settlement_flag_date',
           'settlement_amount', 'settlement_date', 'settlement_percentage', 'settlement_status',
           'settlement_term', 'recoveries', 'collection_recovery_fee', 'funded_amnt',
           'funded_amnt_inv', 'last_pymnt_amnt', 'out_prncp', 'out_prncp_inv',  'total_pymnt',
           'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
           'last_pymnt_d',  'last_credit_pull_d', 'disbursement_method', 'initial_list_status',
           'fico_range_low','last_fico_range_high','last_fico_range_low','sec_app_fico_range_high','sec_app_fico_range_low' ]

potential_features = np.setdiff1d(loan_df.columns.tolist(), exclude)

print(len(potential_features))
print(sorted(potential_features))

loan_df = loan_df[potential_features]

We visualize missingness on the remaining columns and see that many of the columns are new from 2012-08-01 and later. We choose to work with data from this date and beyond since the data from prior to this date is only a small subset of our data.

In [ ]:
def view_missingness_by_issue_d(df):
    missing_values_df = get_missing_values_table(df)
    earliest_date = []
    cols_with_missing_vals = missing_values_df[(missing_values_df.Portion_Missing<1) &
                                           (missing_values_df.Missing_Count >0)].index

    for column in cols_with_missing_vals:
        earliest_date.append(min(loan_df[~loan_df[column].isnull()]['issue_d'])) 

    df = pd.DataFrame({'Column':cols_with_missing_vals,
                      'Earliest issue_d for which column value is not null':earliest_date})
    
    return df.sort_values(by='Earliest issue_d for which column value is not null', ascending=False)

view_missingness_by_issue_d(loan_df).head(15)

Now, we see that there are columns relating to details of applications that have multiple borrowers (co-borrowers). These columns only have non-null values from issue dates of 2017-03-01 and later (or 2015-10-01 and later for 'dti_joint', 'annual_inc_joint', and 'verification_status_joint'), so we remove these columns. We remove all of these columns:

annual_inc_joint, dti_joint, revol_bal_joint, sec_app_chargeoff_within_12_mths, sec_app_collections_12_mths_ex_med, sec_app_earliest_cr_line, sec_app_inq_last_6mths, sec_app_mort_acc, sec_app_mths_since_last_major_derog, sec_app_num_rev_accts, sec_app_open_acc, sec_app_open_act_il, sec_app_revol_util, verification_status_joint]

Note that we still do have the application_type variable, which is a simple binary indicator of whether or not the application had co-borrowers.

In [ ]:
coborrower_cols =  ['annual_inc_joint', 'dti_joint', 'revol_bal_joint',
                    'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med',
                    'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc',
                    'sec_app_mths_since_last_major_derog', 'sec_app_num_rev_accts',
                    'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_revol_util',
                    'verification_status_joint']

potential_features = np.setdiff1d(loan_df.columns.tolist(), coborrower_cols)
loan_df = loan_df[potential_features]

After removing these colummns, we take another look at the remaining columns that have missing values.

In [ ]:
display(view_missingness_by_issue_d(loan_df))

get_missing_values_table(loan_df).head(15)

We notice that many of the columns only have non-null values for issue dates of 2013-04-01 and later, so we remove the samples that have issue_dates prior to this. We see that this only reduces our sample size by a small amount.

In [ ]:
print("Number of rows in dataset: {}".format (loan_df.shape[0]))

loan_df = loan_df[pd.to_datetime(loan_df['issue_d'], format='%b-%Y') >= pd.to_datetime('03/01/2013', format='%m/%d/%Y')]


print("Number of rows in dataset after removing loans issued prior to 2013-04-01: {}"\
      .format (loan_df.shape[0]))

**Taking another look at missingness: **

In [ ]:
display(view_missingness_by_issue_d(loan_df))

get_missing_values_table(loan_df).head(15)

Great! The missingness for the remaining columns is much more reasonable, but there are still a few columns with large portions of missing values. Consulting the data dictionary reveals a plausible explanation for this:

Many of the columns that still have high levels of missingness are those which measure "months since" something happened. For example, months_since_recent_bc_dlq measures "Months since most recent bankcard delinquency." Thus, it's likely that the "missing" values are not truly missing data; rather, the borrower has simply never had a bankcard delinquency. Thus, we choose to process the "month since" columns by filling the null values with the maximum observed value + 1, so that the borrowers who have never had delinquencies just have the largest value for number of months since delinquency. Since we will be using decision-tree based models, this is an appropriate approach that allows the decision tree to separate the borrowers who have had recent delinquencies from those who have not.

We also process a few other columns below:

Re-categorize emp_length so that there are fewer categories
Cast revol_util to float
Change loan_grade and grade values to numerical rankings (i.e., an ordinal variable with the lowest grade/subgrade indicating the least-risky loans)
credit_line_age and earliest_cr_line
int_rate
zip_code
fully_paid and loan_status

In [ ]:
print('emp_length value counts: \n{}\n'.format(loan_df.emp_length.value_counts()))

In [ ]:
def process_emp_length(df):
    df_new = df.copy()
    #0-1 years inclues: null, None, < 1 year, 1 year
    df_new['emp_length'] = df_new['emp_length'].fillna('0-1 years')
    df_new['emp_length'] = np.where(df_new['emp_length'].isin([np.nan,None,'< 1 year','1 year']),
                                     '0-1 years',df_new['emp_length'])
    #2-4 years inclues: 2 years, 3 years, 4 years
    df_new['emp_length'] = np.where(df_new['emp_length'].isin(['2 years','3 years','4 years']),
                                     '2-4 years',df_new['emp_length'])
    
    #5-9 years inclues: 5 years, 6 years, 7 years, 8 years, 9 years
    df_new['emp_length'] = np.where(df_new['emp_length'].isin(['5 years','6 years','7 years','8 years','9 years']),
                                     '5-9 years',df_new['emp_length'])
    #10+ years includes 10+ years (no change)
    return df_new

def process_revol_util(df):
    df_new = df.copy()
    
    df_new['revol_util'] = df_new['revol_util'].fillna(-100)
    df_new['revol_util'] = df_new['revol_util'].apply(lambda x: float(str(x).split('%')[0])/100)
    df_new['revol_util'] = np.where(df_new['revol_util']==-1.0,np.nan,df_new['revol_util'])
    
    return df_new

def process_month_since_cols(df):
    df_new = df.copy()

    mo_since_cols = ['mo_sin_old_il_acct','mths_since_last_delinq','mths_since_last_major_derog',
                     'mths_since_last_record','mths_since_recent_bc_dlq','mths_since_recent_inq',
                     'mths_since_recent_revol_delinq','mo_sin_old_rev_tl_op','mths_since_recent_bc',
                     'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl']

    for col in mo_since_cols:
        df_new[col].fillna(df_new[col].max()+1, inplace=True)
    
    return df_new

def process_loan_grades(df):
    df_new = df.copy()
    
    # turn sub_grade into ordinal mapping
    sorted_subgrades = sorted(df.sub_grade.unique())
    subgrade_dict = dict(zip(sorted_subgrades, range(len(sorted_subgrades))))
    df_new['sub_grade'] = df_new['sub_grade'].map(subgrade_dict)
    
    # turn grade into ordinal mapping
    sorted_grades = sorted(df.grade.unique())
    grade_dict = dict(zip(sorted_grades, range(len(sorted_grades))))
    df_new['grade'] = df_new['grade'].map(grade_dict)
    return df_new
    
    
def process_loan_cols(df):
    df_processed = process_emp_length(df)
    df_processed = process_revol_util(df_processed)
    df_processed = process_month_since_cols(df_processed)
    df_processed = process_loan_grades(df_processed)
    
    #add credit_line_age
    df_processed['issue_d'] = pd.to_datetime(df_processed['issue_d'], format='%b-%Y')
    df_processed['earliest_cr_line'] = pd.to_datetime(df_processed['earliest_cr_line'])
    df_processed['credit_line_age'] = df_processed['issue_d'] - pd.to_datetime(df_processed['earliest_cr_line'])
    df_processed = df_processed.drop(columns='earliest_cr_line')

    #process int_rate, zip_code, credit_line_age
    df_processed['int_rate'] = df_processed['int_rate'].apply(lambda x: float(str(x).split('%')[0])/100)
    df_processed['zip_code'] = df_processed['zip_code'].apply(lambda x: x[:3])
    df_processed['credit_line_age'] = df_processed['credit_line_age'].apply(lambda x: x.days)
    
    # generate new column for outcome variable ('fully_paid'); drop 'loan_status'
    df_processed['fully_paid'] = df_processed['loan_status'].map({'Fully Paid':1, 'Charged Off':0})
    df_processed = df_processed.drop(columns='loan_status')
    
    return df_processed

loan_df = process_loan_cols(loan_df)

Missing Values: Imputation for Continuous Columns
We will fill the remaining columns' missing values using linear regression model-based imputation. First, we verify that all of the remaining columns are continuous float-64 variables for which this model imputation is appropriate.

Then, we use pd.get_dummies() to get dummy-variables for the necessary categorical columns. This is an important step which should be done prior to model-based imputation

Finally, we proceed with our imputation.

In [ ]:
missing_data = get_missing_values_table(loan_df)
display(missing_data[missing_data.Missing_Count!=0])
missing_cols = missing_data[missing_data.Missing_Count!=0].index

print("datatypes of the columns that still have missing values: ")
print(loan_df[missing_cols].dtypes.unique().tolist())

Before performing imputation, we need to process some of the categorical variables using pd.get_dummies. We do so below.

In [ ]:
#Rerun Types
types_df = pd.DataFrame(loan_df.dtypes,columns=['Types'])
display(types_df[types_df['Types']!='float64'].sort_values(by='Types'))


object_vars=loan_df.select_dtypes(include='object').columns.tolist()
object_vars = np.setdiff1d(object_vars,['addr_state', 'zip_code'])
print(object_vars)

#Dummy encoding
loan_df = pd.get_dummies(loan_df,columns=object_vars,drop_first=True) 

In [ ]:
#columns where imputation is neededpd.
missing_cols = loan_df.columns[loan_df.isnull().any()].tolist()

#verify they're all continuous dtypes appropriate for linear regression imputation
loan_df[missing_cols].dtypes.unique() # all float64

Performming Linear Regression Model-Based Imputation

In [ ]:
def impute_missing_continuous(df, missing_columns, cols_to_exclude):
    clean_df = df.copy()
    
    for column in missing_columns:
        types_df = pd.DataFrame(df.dtypes, columns=['Types'])
        
        # Remove response variable
        impute_train = clean_df[~clean_df[column].isnull()].drop(columns=cols_to_exclude).sample(frac=0.1, random_state=12)
        
        # Split target
        X_impute_train = impute_train.drop(columns=column)
        Y_impute_train = impute_train[column]
        
        # Mean Imputation for current nulls for columns that did not get imputed yet
        X_impute_train = X_impute_train.fillna(X_impute_train.mean())
        
        # Train LinearRegression
        impute_ols = LinearRegression(fit_intercept=True)
        impute_ols.fit(X_impute_train, Y_impute_train)
        
        # Generate new temp column with model predictions
        # Only replace rows where the value is null with the predicted value
        exclude_cols = cols_to_exclude + [column]
        predictions = clean_df.drop(columns=exclude_cols)
        
        # Mean Imputation for current nulls for columns that did not get imputed yet
        predictions = predictions.fillna(predictions.mean())
        clean_df['temp'] = impute_ols.predict(predictions)
        
        clean_df[column] = np.where(clean_df[column].isnull(), 
                                  clean_df['temp'], 
                                  clean_df[column])
        clean_df = clean_df.drop(columns='temp')
    
    return clean_df

# Call the function
clean_df = impute_missing_continuous(
    loan_df, 
    missing_columns=missing_cols, 
    cols_to_exclude=['fully_paid', 'issue_d', 'zip_code', 'addr_state']
)

In [ ]:
clean_df.isnull().values.any()

In [ ]:
print("float64 columns: ")
print(clean_df.dtypes[clean_df.dtypes=='float64'].index.tolist())

print("\nnon-float64 columns: ")
print(clean_df.dtypes[clean_df.dtypes!='float64'])

print("\n\nVisualizing unique values for non-float64 variables (except for zip_code and issue_d)")
for col in clean_df.dtypes[clean_df.dtypes!='float64'].index.tolist():
    if not col in['issue_d', 'zip_code']:
        print("\n", col)
        print(clean_df[col].unique())

In [ ]:
clean_df.to_pickle('clean_df_5pct_subset.pkl')

## Data Clean on Entire Dataset

In [ ]:
df = pd.read_csv('accepted_2007_to_2018Q3.csv.gz')
loan_df = df[df['loan_status'].isin(['Charged Off', 'Fully Paid'])]

exclude = ['id', 'member_id', 'url', 'next_pymnt_d', 'pymnt_plan', 'out_prncp',
           'out_prncp_inv', 'policy_code', 'open_acc_6m', 'open_act_il', 'open_il_12m',
           'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
           'inq_fi', 'total_cu_tl', 'inq_last_12m', 'il_util', 'all_util','mths_since_rcnt_il',
           'num_tl_120dpd_2m', 'num_tl_30dpd', 'deferral_term', 'hardship_amount',
           'hardship_dpd', 'hardship_end_date', 'hardship_flag', 'hardship_last_payment_amount',
           'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount',
           'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type',
           'orig_projected_additional_accrued_interest', 'payment_plan_start_date',
           'desc', 'title', 'emp_title', 'debt_settlement_flag', 'debt_settlement_flag_date',
           'settlement_amount', 'settlement_date', 'settlement_percentage', 'settlement_status',
           'settlement_term', 'recoveries', 'collection_recovery_fee', 'funded_amnt',
           'funded_amnt_inv', 'last_pymnt_amnt', 'out_prncp', 'out_prncp_inv',  'total_pymnt',
           'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
           'last_pymnt_d',  'last_credit_pull_d', 'disbursement_method', 'initial_list_status',
           'annual_inc_joint', 'dti_joint', 'revol_bal_joint','sec_app_chargeoff_within_12_mths',
           'sec_app_collections_12_mths_ex_med','sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
           'sec_app_mort_acc','sec_app_mths_since_last_major_derog', 'sec_app_num_rev_accts',
           'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_revol_util',
           'verification_status_joint',
           'fico_range_high','fico_range_low','last_fico_range_high','last_fico_range_low','sec_app_fico_range_high','sec_app_fico_range_low']

potential_features = np.setdiff1d(loan_df.columns.tolist(), exclude)
loan_df = loan_df[potential_features]

loan_df = loan_df[pd.to_datetime(loan_df['issue_d'], format='%b-%Y') >= pd.to_datetime('03/01/2013', format='%m/%d/%Y')]

loan_df = process_loan_cols(loan_df)

object_vars = loan_df.select_dtypes(include='object').columns.tolist()
vars_for_dummies = np.setdiff1d(object_vars,['addr_state', 'zip_code'])

tmp_df = loan_df[vars_for_dummies]
loan_df = pd.get_dummies(loan_df,columns=vars_for_dummies,drop_first=True) 

missing_cols = loan_df.columns[loan_df.isnull().any()].tolist()

clean_df = impute_missing_continuous(loan_df, missing_columns=missing_cols,
                                     cols_to_exclude=['fully_paid','issue_d', 'zip_code', 'addr_state'])

for col in vars_for_dummies:
    clean_df[col] = tmp_df[col]
    
print(sorted(clean_df.columns.tolist()))

In [ ]:
print(clean_df.shape)

In [ ]:
clean_df.isnull().values.any()

In [ ]:
clean_df.to_pickle('clean_df_for_eda.pkl')

In [ ]:
clean_df.groupby(['fully_paid']).size()

## Subset Data

In [ ]:
shorten_df = pd.read_pickle('clean_df_for_eda.pkl')

In [ ]:
# Subset while maintaining distribution.

subset_size = 200000

proportions = shorten_df['fully_paid'].value_counts(normalize=True)
sample_sizes = (proportions * subset_size).round().astype(int)

shorten_df = (
    shorten_df.groupby('fully_paid', group_keys=False)
    .apply(lambda group: group.sample(n=sample_sizes[group.name], random_state=42))
)

In [ ]:
def CapOutliers(df,columns_to_cap):
    for column in columns_to_cap:
        Q1 = df[column].quantile(.25)
        Q3 = df[column].quantile(.75)
        IQR = Q3 - Q1
        upper = (Q3 + 1.5 * IQR)
        lower = (Q1 - 1.5 * IQR)

        df[column] = np.where(df[column] > upper,upper,np.where(df[column] < lower,lower,df[column]))

        return df

In [ ]:
columns_to_cap = ['acc_now_delinq', 'acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 'dti', 'inq_last_6mths', 'installment', 'int_rate', 'loan_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'revol_bal', 'revol_util', 'tax_liens', 'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_acc', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'total_rev_hi_lim']

loan_df = CapOutliers(shorten_df,columns_to_cap)

In [ ]:
df_encoded = pd.get_dummies(loan_df,columns=['application_type','emp_length','home_ownership','purpose','term','verification_status','addr_state'])

In [ ]:
X = df_encoded.drop(['fully_paid','zip_code','issue_d','int_rate','installment','grade','sub_grade'],axis=1)
y = df_encoded['fully_paid']

# XGBoost Model

In [ ]:
# Import Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import PartialDependenceDisplay
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Step 2: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 3: Train an initial XGBoost model
initial_model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    eval_metric='logloss'
)
initial_model.fit(X_train_scaled, y_train)

In [ ]:
# Step 4: Perform feature selection
selection = SelectFromModel(initial_model, prefit=True, threshold='mean')
X_train_selected = selection.transform(X_train_scaled)
X_test_selected = selection.transform(X_test_scaled)

In [ ]:
# Step 5: Define a parameter grid for hyperparameter tuning
param_grid = {
    'eval_metric': ['logloss'],
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5],
    'min_child_weight': [1, 3],
    'subsample': [0.5, 0.9],
    'colsample_bytree': [0.5, 0.8],
    'scale_pos_weight': [1, 3]
}

In [ ]:
# Step 6: Perform hyperparameter tuning
model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1', 
    cv=3,
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train_selected, y_train)

In [ ]:
# Step 7: Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)
y_pred_proba = best_model.predict_proba(X_test_selected)[:, 1]

xgboost_metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1': f1_score(y_test, y_pred),
    'roc_auc': roc_auc_score(y_test, y_pred_proba),
    'best_params': grid_search.best_params_
}

print("Evaluation Metrics:")
for metric, value in xgboost_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f}" if isinstance(value, float) else f"{metric.capitalize()}: {value}")



In [ ]:
# Step 8: Visualize feature importance
plt.figure(figsize=(10, 8))
xgb.plot_importance(best_model, importance_type='weight', max_num_features=20)
plt.title("Feature Importance from XGBoost")
plt.show()


In [ ]:
from math import ceil

# Convert selected features back to their original names
selected_features = X.columns[selection.get_support()]

# Determine the number of rows and columns based on the number of features
num_features = len(selected_features)
num_cols = 2  # You can adjust the number of columns as desired
num_rows = ceil(num_features / num_cols)

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
axes = axes.flatten()

# Iterate over all selected features
for i, feature in enumerate(selected_features):
    PartialDependenceDisplay.from_estimator(
        best_model, X_test_selected, features=[i], feature_names=selected_features, ax=axes[i]
    )

# Hide any unused subplots if there are more subplots than features
for j in range(num_features, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()


## Logistic Regression

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from tools import display_dataframe_to_user



In [ ]:
# Step 1: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Step 2: Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 3: Define Hyperparameter Grid
lr_params = {
    'C': [0.01, 0.1, 1, 10],  # Regularization strength
    'penalty': ['l2'],        # L2 regularization
    'solver': ['lbfgs']       # Optimization solver
}

In [ ]:
# Step 4: Hyperparameter Tuning with GridSearchCV
log_reg = LogisticRegression(random_state=42, max_iter=500)
grid_search_lr = GridSearchCV(
    estimator=log_reg,
    param_grid=lr_params,
    scoring='f1',  # Optimize for F1-score
    cv=3,          # 3-fold cross-validation
    verbose=1,
    n_jobs=-1      # Use all available CPU cores
)
grid_search_lr.fit(X_train_scaled, y_train)

In [ ]:
# Step 5: Get the Best Model and Parameters
best_log_reg = grid_search_lr.best_estimator_
best_params = grid_search_lr.best_params_
print(f"Best Parameters: {best_params}")

In [ ]:
# Step 6: Make Predictions
y_pred = best_log_reg.predict(X_test_scaled)
y_pred_proba = best_log_reg.predict_proba(X_test_scaled)[:, 1]

In [ ]:
# Step 7: Compute Evaluation Metrics

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Create the dictionary
logistic_metrics = {
    "Accuracy": accuracy,
    "Precision (Class 1)": precision,
    "Recall (Class 1)": recall,
    "F1-Score (Class 1)": f1,
    "AUC (ROC)": roc_auc
}

print("\nLogistic Regression Metrics After Hyperparameter Tuning:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")


In [ ]:
# Step 8: Generate Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
# Step 9: Plot ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], "k--", label="Random Guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Step 10: Feature Importance (Coefficient Magnitudes)
feature_importances = pd.DataFrame({
    "Feature": X.columns,
    "Importance": abs(best_log_reg.coef_[0])
}).sort_values(by="Importance", ascending=False)

print("\nTop 10 Features by Importance:")
print(feature_importances.head(10))

# Visualize feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importances["Feature"].head(10), feature_importances["Importance"].head(10))
plt.gca().invert_yaxis()
plt.title("Top 10 Features by Importance")
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Step 11: Partial Dependence Plots for Top Features
# Selecting top features
top_features = feature_importances["Feature"].head(3).values

fig, axes = plt.subplots(len(top_features), 1, figsize=(10, 10))
for i, feature in enumerate(top_features):
    PartialDependenceDisplay.from_estimator(
        best_log_reg, X_test_scaled, features=[i], feature_names=X.columns, ax=axes[i]
    )
plt.tight_layout()
plt.show()

## Compare Results

In [ ]:
# Create comparison DataFrame
comparison = pd.DataFrame([
    {"Metric": "Accuracy", "Logistic Regression": logistic_metrics["Accuracy"], "XGBoost": xgboost_metrics["Accuracy"]},
    {"Metric": "AUC (ROC)", "Logistic Regression": logistic_metrics["AUC (ROC)"], "XGBoost": xgboost_metrics["AUC (ROC)"]},
    {"Metric": "Precision (Class 1)", "Logistic Regression": logistic_metrics["Precision (Class 1)"], "XGBoost": xgboost_metrics["Precision (Class 1)"]},
    {"Metric": "Recall (Class 1)", "Logistic Regression": logistic_metrics["Recall (Class 1)"], "XGBoost": xgboost_metrics["Recall (Class 1)"]},
    {"Metric": "F1-Score (Class 1)", "Logistic Regression": logistic_metrics["F1-Score (Class 1)"], "XGBoost": xgboost_metrics["F1-Score (Class 1)"]}
])

# Display the DataFrame
print("Performance Summary: Logistic Regression vs. XGBoost")
print(comparison)
